In [12]:
from datasets import load_dataset

# OpenWebTextデータセットのロード
dataset = load_dataset('openwebtext')

In [13]:
# 'train'セットから5000サンプルを取得
limited_dataset = dataset['train'].select(range(5000))

# データの確認
print(limited_dataset)


Dataset({
    features: ['text'],
    num_rows: 5000
})


In [17]:
from transformers import GPT2Tokenizer
# トークナイザーをロード
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [18]:
context_length = 128

In [29]:
import torch
from transformers import GPT2LMHeadModel, AutoConfig


# GPT-2の設定を作成（小型モデル用）
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
) # "gpt2"を指定して小型モデルを取得
# モデルを初期化
student_model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in student_model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")


GPT-2 size: 124.4M parameters


In [22]:
# デバイスの設定
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# トークナイザーと教師モデルをロード
teacher_model = GPT2LMHeadModel.from_pretrained('gpt2-large')
model_size = sum(t.numel() for t in teacher_model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

teacher_model.to(device)  # GPUを使用する場合
student_model.to(device)




GPT-2 size: 774.0M parameters


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [23]:
tokenizer.pad_token = tokenizer.eos_token

In [24]:
import torch.nn.functional as F
def distillation_loss(student_logits, teacher_logits, temperature=2.0):
    teacher_probs = F.softmax(teacher_logits / temperature, dim=-1)
    student_probs = F.log_softmax(student_logits / temperature, dim=-1)
    return F.kl_div(student_probs, teacher_probs) * (temperature ** 2)

In [25]:
from tqdm import tqdm
import torch.optim as optim

# オプティマイザーの設定
optimizer = optim.AdamW(student_model.parameters(), lr=5e-5)

num_epochs = 3  # エポック数
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    epoch_progress = tqdm(limited_dataset, desc="Batch Progress", leave=False)
    
    for example in epoch_progress:
        input_text = example['text']
        inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True).to('cuda')

        # 教師モデルの出力を取得
        with torch.no_grad():
            teacher_output = teacher_model(**inputs)
            teacher_logits = teacher_output.logits

        # 生徒モデルの出力を取得
        student_output = student_model(**inputs)
        student_logits = student_output.logits

        # 蒸留損失の計算
        loss = distillation_loss(student_logits, teacher_logits)
        
        # 勾配計算とオプティマイザーのステップ
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 各バッチの損失を進捗バーに表示
        epoch_progress.set_postfix({"Loss": loss.item()})
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

Epoch 1/3


Batch Progress:   0%|          | 0/5000 [00:00<?, ?it/s]/root/miniconda3/envs/my-env/lib/python3.9/site-packages/torch/nn/functional.py:3369: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch 1/3, Loss: 9.94927468127571e-05
Epoch 2/3


Epoch 2/3, Loss: 8.963559230323881e-05
Epoch 3/3


Epoch 3/3, Loss: 8.472816261928529e-05


In [ ]:
# 評価関数の定義
def evaluate_model(model, tokenizer, input_texts, device):
    model.eval()  # 評価モード
    total_loss = 0
    total_perplexity = 0
    count = 0
    with torch.no_grad():
        for text in input_texts:
            # 入力をデバイスに移動
            inputs = tokenizer(text, return_tensors='pt').to(device)
            outputs = model(**inputs, labels=inputs["input_ids"])  # 入力をモデルに渡す
            loss = outputs.loss.item()
            total_loss += loss
            perplexity = torch.exp(torch.tensor(loss))
            total_perplexity += perplexity.item()
            count += 1

    # 平均損失とパープレキシティを計算
    avg_loss = total_loss / count if count > 0 else float('inf')
    avg_perplexity = total_perplexity / count if count > 0 else float('inf')
    return avg_loss, avg_perplexity

# 使用例
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 評価データを定義
input_texts = ["This is a test sentence.", "How are you today?"]

# 教師モデルの評価
teacher_loss, teacher_perplexity = evaluate_model(teacher_model, tokenizer, input_texts, device)
print(f"Teacher Model - Loss: {teacher_loss:.4f}, Perplexity: {teacher_perplexity:.4f}")

# 生徒モデルの評価
student_loss, student_perplexity = evaluate_model(student_model, tokenizer, input_texts, device)
print(f"Student Model - Loss: {student_loss:.4f}, Perplexity: {student_perplexity:.4f}")

Teacher Model - Loss: 3.3081, Perplexity: 33.6072
Student Model - Loss: 5.7662, Perplexity: 352.4743


In [28]:
student_model.save_pretrained('distill_GPTlm_model')

In [30]:
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
) # "gpt2"を指定して小型モデルを取得
# モデルを初期化
no_train_model = GPT2LMHeadModel(config)

no_train_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [32]:
no_train_loss, no_train_perplexity = evaluate_model(no_train_model, tokenizer, input_texts, device)
print(f"No train Model - Loss: {no_train_loss:.4f}, Perplexity: {no_train_perplexity:.4f}")

No train Model - Loss: 11.2473, Perplexity: 78959.7949
